In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# After following the installation (or building from source) instructions,
# you should be able to import jaxquantum without having to add it to
# your path explicitly. 

import jaxquantum as jqt
from jax import device_put, jit
import jax.numpy as jnp

In [4]:
omega_q = device_put(5.0) #GHz
Omega = device_put(.1)
H0 = omega_q/2.0*jqt.sigmaz()
H1 = Omega*jqt.sigmax()
g_state = jqt.ket2dm(jqt.basis(2,0)) + 0.0j

c_ops = jnp.array([])

In [5]:

@jit
def pi_exp(omega_d):    
    t_list = jnp.linspace(0,jnp.pi/Omega,101)
    def H_func(rho, t, H0_val, H1_val, c_ops_val):
        fac = jnp.cos((omega_d+0j)*t)
        H = H0 + H1*fac
        rho_dot = -1.0j * (H @ rho)
        rho_dot += -1.0j * (-rho @ H)
        return rho_dot
    
    states = odeint(H_func, g_state, t_list, H0, H1, c_ops)
    final_state = states[-1]
    return jnp.real(jnp.trace(final_state @ jqt.sigmaz()))

In [6]:
@jit
def Ht(t):
    fac = jnp.cos((omega_q+0j)*t)
    return H0 + H1*fac

In [9]:
%timeit results = jqt.mesolve(g_state, jnp.linspace(0,jnp.pi/Omega,101), c_ops=c_ops, H0=H0, use_density_matrix=False) 

1.07 ms ± 69.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
results = jqt.mesolve(g_state, jnp.linspace(0,jnp.pi/Omega,101), c_ops=c_ops, H0=H0, use_density_matrix=False) 